In [3]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Load the LFW dataset
from sklearn.datasets import fetch_lfw_people
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)


X_train, X_test, y_train, y_test = train_test_split(lfw_people.data, lfw_people.target, test_size=0.25, random_state=42)




In [8]:
!pip uninstall opencv-contrib-python
!pip install opencv-contrib-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 MB 1.2 MB/s eta 0:00:0000:0100:02


In [9]:
classifiers = {
    "Eigenfaces": cv2.face_EigenFaceRecognizer_create(),
    "Fisherfaces": cv2.face_FisherFaceRecognizer_create(),
    "LBPH": cv2.face_LBPHFaceRecognizer_create(),
    "Haarcascades": cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
}


AttributeError: module 'cv2' has no attribute 'face_EigenFaceRecognizer_create'

In [ ]:
for clf_name, clf in classifiers.items():
    if clf_name == "Haarcascades":
        continue
    clf.train(X_train, y_train)


In [ ]:
face_cascade = classifiers["Haarcascades"]

# Lists to store results
true_labels = []
predicted_labels = []

# Perform face recognition
for i in range(len(X_test)):
    # Haarcascades detection
    faces = face_cascade.detectMultiScale(X_test[i], scaleFactor=1.1, minNeighbors=5)
    
    if len(faces) > 0:
        x, y, w, h = faces[0]
        face_roi = X_test[i][y:y+h, x:x+w]

        # Eigenfaces, Fisherfaces, and LBPH recognition
        for clf_name, clf in classifiers.items():
            if clf_name == "Haarcascades":
                continue
            label, _ = clf.predict(face_roi)
            true_labels.append(y_test[i])
            predicted_labels.append(label)


In [ ]:
for clf_name, clf in classifiers.items():
    if clf_name == "Haarcascades":
        continue
    accuracy = accuracy_score(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, target_names=lfw_people.target_names)
    print(f"Classifier: {clf_name}")
    print(f"Accuracy: {accuracy}")
    print(report)